In [3]:
import boto3
import pandas as pd
import json
from io import StringIO

In [4]:
# Initialize S3 client
s3 = boto3.client("s3")

# Define bucket and keys
bucket_name = "rearc-data-quest-vidushi"
key_csv = "bls/pr/pub/time.series/pr/pr.data.0.Current"  # CSV file from Part 1


In [11]:
# Function to load CSV data from S3 and read raw content
def load_s3_csv_raw(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    csv_data = response["Body"].read().decode("utf-8")
    return csv_data

# # Load raw CSV data from Part 1
# raw_csv_data = load_s3_csv_raw(bucket_name, key_csv)
# print("Raw CSV Data:")
# print(raw_csv_data[:500])  # Print the first 500 characters to inspect the format

# Load CSV data as DataFrame with tab separator
df_csv = pd.read_csv(StringIO(raw_csv_data), sep="\t")
print("CSV DataFrame:")
print(df_csv.head())

CSV DataFrame:
   series_id          year period         value footnote_codes
0  PRS30006011        1995    Q01           2.6            NaN
1  PRS30006011        1995    Q02           2.1            NaN
2  PRS30006011        1995    Q03           0.9            NaN
3  PRS30006011        1995    Q04           0.1            NaN
4  PRS30006011        1995    Q05           1.4            NaN


In [9]:
# Function to get the latest JSON file from a prefix in S3
def get_latest_json_key(bucket, prefix):
    paginator = s3.get_paginator("list_objects_v2")
    latest_key = None
    latest_date = None

    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            key = obj["Key"]
            last_modified = obj["LastModified"]
            if latest_date is None or last_modified > latest_date:
                latest_key = key
                latest_date = last_modified

    return latest_key

In [10]:

# Get the latest JSON file from Part 2
latest_json_key = get_latest_json_key(bucket_name, "bls/api/")
print(f"Latest JSON Key: {latest_json_key}")

# Load JSON data from Part 2
def load_s3_json_to_df(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    data = json.loads(response["Body"].read().decode("utf-8"))
    return pd.DataFrame(data["data"])

df_json = load_s3_json_to_df(bucket_name, latest_json_key)
print("JSON DataFrame:")
print(df_json.head())

Latest JSON Key: bls/api/acs_population_20260207T114446Z.json
JSON DataFrame:
  Nation ID         Nation  Year   Population
0   01000US  United States  2013  316128839.0
1   01000US  United States  2014  318857056.0
2   01000US  United States  2015  321418821.0
3   01000US  United States  2016  323127515.0
4   01000US  United States  2017  325719178.0


### 1. Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

In [12]:

# Filter data for years 2013 to 2018 (inclusive)
filtered_df = df_json[(df_json["Year"] >= 2013) & (df_json["Year"] <= 2018)]

# Calculate mean and standard deviation of the Population column
mean_population = filtered_df["Population"].mean()
std_population = filtered_df["Population"].std()

print(f"Mean Population (2013-2018): {mean_population}")
print(f"Standard Deviation of Population (2013-2018): {std_population}")

Mean Population (2013-2018): 322069808.0
Standard Deviation of Population (2013-2018): 4158441.040908095


### 2. Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year.

In [15]:
# Clean column names: convert to lowercase and remove leading/trailing whitespace
df_csv.columns = df_csv.columns.str.strip().str.lower()
print("Cleaned column names:")
print(df_csv.columns.tolist())

Cleaned column names:
['series_id', 'year', 'period', 'value', 'footnote_codes']


In [18]:
# Convert 'value' and 'year' columns to numeric types
df_csv['value'] = pd.to_numeric(df_csv['value'], errors='coerce')
df_csv['year'] = pd.to_numeric(df_csv['year'], errors='coerce')

# Remove rows only if all columns are null
df_csv = df_csv.dropna(how='all')

# Strip leading and trailing spaces from all string columns
df_csv = df_csv.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

print("Data cleaned successfully!")
print(f"Shape: {df_csv.shape}")
print(df_csv.head())

Data cleaned successfully!
Shape: (37521, 5)
     series_id  year period  value footnote_codes
0  PRS30006011  1995    Q01    2.6            NaN
1  PRS30006011  1995    Q02    2.1            NaN
2  PRS30006011  1995    Q03    0.9            NaN
3  PRS30006011  1995    Q04    0.1            NaN
4  PRS30006011  1995    Q05    1.4            NaN


In [19]:
# Group data by series_id and year, then calculate the sum of "value" for each group
grouped = df_csv.groupby(["series_id", "year"])['value'].sum().reset_index()

# Find the best year for each series_id
best_years = grouped.loc[grouped.groupby("series_id")['value'].idxmax()]

# Rename columns for clarity
best_years.rename(columns={"year": "best_year", "value": "max_value"}, inplace=True)

# Display the report
print("Best Year Report:")
print(best_years)

Best Year Report:
        series_id  best_year  max_value
27    PRS30006011       2022     20.500
58    PRS30006012       2022     17.100
65    PRS30006013       1998    705.895
108   PRS30006021       2010     17.700
139   PRS30006022       2010     12.400
...           ...        ...        ...
8459  PRS88003192       2002    282.800
8512  PRS88003193       2024    862.564
8541  PRS88003201       2022     38.900
8572  PRS88003202       2022     29.700
8605  PRS88003203       2024    590.619

[282 rows x 3 columns]


### 3. Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset).

In [23]:
# Filter df_csv for the specified series_id and period
filtered_timeseries = df_csv[(df_csv["series_id"] == "PRS30006032") & (df_csv["period"] == "Q01")]

# Merge with population dataframe on 'year'
report = pd.merge(
    filtered_timeseries,
    df_json[["Year", "Population"]],
    left_on="year",
    right_on="Year",
    how="left"
)

# Select relevant columns and rename for clarity
report = report[["series_id", "year", "period", "value", "Population"]]

print("Report for series_id = PRS30006032 and period = Q01:")
report

Report for series_id = PRS30006032 and period = Q01:


,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.2,NaN
2,PRS30006032,1997,Q01,2.8,NaN
3,PRS30006032,1998,Q01,0.9,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.3,NaN
7,PRS30006032,2002,Q01,-6.6,NaN
8,PRS30006032,2003,Q01,-5.7,NaN
9,PRS30006032,2004,Q01,2.0,NaN
